# **<center>SPARC FAIR Codeathon 2022</center>**
<center>
<a href="https://sparc.science">
<img src="https://sparc.science/_nuxt/img/logo-sparc-wave-primary.8ed83a5.svg" alt="SPARC" width="150"/>
</a>
</center>
<center>
<a href="https://sparc.science/help/2022-sparc-fair-codeathon">
<img src="https://images.ctfassets.net/6bya4tyw8399/2qgsOmFnm7wYIfRrPrqbgx/ae3255858aa12bfcebb52e95c7cacffe/codeathon-graphic.png" alt="FAIR" width="75">
</a>
</center>

## <center>Mapping 2D data to a 3D organ scaffold</center>


## **Introduction**
Welcome to the first of the Quilted Tutorials! We will be demonstrating different features from the [**SPARC**](https://sparc.science/) project. The goal will be to project the 2D locations of neurites in the rat stomach onto a 3D scaffhold of the organ. The data points and the 3D scaffhold will be pulled from **SPARC** datasets. Because the data is [**FAIR**](https://www.nature.com/articles/sdata201618) we will be combining three different datasets of the spatial distribution of the vagal afferents and efferents. Here is the workflow for this tutorial

![workflow](img/workflow.png)

## **Installing the dependencies**
This tutorial relies on several Python packages that have been developed as part of the **SPARC** project. We will be installing them in order to complete this tutorial.

In [26]:
!pip install pandas # To load SPARC datasets in Python
!pip install openpyxl # Pandas complement for Microsoft Excel files
!pip install ipywidgets # To interact with plotted data
# For data visualisation
!pip install numpy
!pip install numpy-stl
!pip install matplotlib
!pip install ipympl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## **Retrieving the data**
Now that all the dependencies have been installed we will retrieve the data from directly from the [**SPARC**](https://sparc.science) project website. 
We will be using three datasets:
 * [vagal afferents associated with the myenteric plexus of the rat stomach](https://sparc.science/datasets/10?type=dataset&datasetDetailsTab=files)
 * [vagal afferents within the longitudinal and circular muscle layers of the rat stomach](https://sparc.science/datasets/11?type=dataset&datasetDetailsTab=files)
 * [vagal efferents associated with the myenteric plexus of the rat stomach](https://sparc.science/datasets/12?type=dataset&datasetDetailsTab=files)
 
You can search through all of the **SPARC** datasets [here](https://sparc.science/data?type=dataset) or simply click on the links above to be redirected directly to the datasets. 

It is possible to downlowd the entire dataset by clicking on the purple ***Download full dataset*** button  in the **Download Dataset** tab or selecting specific files and folders in the **Dataset Files** tab lower in the page. If you haven't used the links above, you can click on the purple ***Get Dataset*** button on the left side of the screen or directly in the ***Files*** tab. 

For this tutorial, we are only interested in the contents of the _derative_ folder which contains two .xlsx files: one with the data (IGLE_data.xlsx, IMA_analyzed_data.xlsx, and Efferent_data.xlsx) and a manifest (manifest.xlsx). Enter the _derivative_ folder and select the xlsx file containing the data by ticking the box in front of it. Download the file by clicking the **Download Selected Files and Folders** button at the bottom. You will then be prompted to select the location in which to save it. For each dataset, save it in the _SPARC-tutorial_ folder. 

#### ⚠️  **SPARC Guru tip**: 
Ever heard of Pennsieve? It's the **SPARC** tool to use if you want to avoid downloading the data manually. It even has a Python API so you can integrate it directly into this Notebook! Check out the [documentation](https://docs.pennsieve.io/).

### **Imports**
Here we import all of the dependencies that we will need to run the code correctly.

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from stl import mesh as msh
from mpl_toolkits import mplot3d
from ipywidgets import interact, Checkbox, fixed

### **Helper functions**
Now that we have installed and imported the required dependencies, we are going to define some helper functions.

#### _get\_position_
This function will allow use to convert the position of the data from a percentage into a distance in mm.

In [28]:
def get_position(percent, min_val, max_val):
    """ Converts the position from percentage to distance.
    
    Inputs:
    percent -- float, percentage value.
    min_val -- float, minimum distance for conversion.
    max_val -- float, maximum distance for conversion.
    
    Outputs:
    converted_value -- float, converted value.
    
    """
    return percent / 100 * (max_val - min_val) + min_val 

#### _load\_data_
This function will allow use to extract the correct elements inside the data files and store them into a data frame.

In [29]:
def load_data(data_name, col_keeps, y_lims, z_lims):
    """ Loads the data from an .xlsx file.
    
    Inputs:
    data_name -- str, nane of the .xlsx file to read.
    col_keeps -- dict{str:str}, dictionnary containing the names of the columns
        to keep.
    y_lims -- list[int], limits for the y direction to convert back to mm,
            first element is the minimum and second is the maximum.
    z_lims -- list[int], limits for the z direction to convert back to mm,
        first element is the minimum and second is the maximum.
    
    Outputs:
    df -- DataFrame, data frame containing the desired data.
    
    """
    df = pd.read_excel(data_name)
    # remove unnecessary columns
    for col in df.columns:
        if col in col_keeps:
            df.rename(columns = {col:col_keeps[col]}, inplace = True)
        else:
            df.drop(col, axis=1, inplace=True)
    df['y'] = get_position(df['%y'], y_lims[0], y_lims[1])
    df['z'] = get_position(df['%x'], z_lims[0], z_lims[1]) # x becomes z
    df['-%y'] = 100 - df['%y']
    # change the area to mm
    return df

#### _convert\_2D\_data_
This function will convert the data from a DataFrame into numpy arrays that will be used for mapping.

In [30]:
def convert_2D_data(df):
    data_array = np.array(df[['y','z','area']])

    # Convert input data to numpy coordinate array and intensity array.
    data_2_dim = np.array(data_array[:, (0,1)])
    data_intensity = data_array[:,2]
    
    return data_2_dim, data_intensity

#### _map\_to\_3D_
This function will map the 2D data to the 3D mesh.

In [31]:
def map_to_3D(input_pts, input_mesh):
    """ Maps an Nx2 numpy array of points in the y-z plane to have an
    x coordinate based on the nearest point in the y-z plane on the input mesh.
    
    Inputs:
    input_pts -- np.array(N, 2) with y coordinates in column 0, and
        z coordinates in column 1.
    input_mesh -- numpy-stl mesh object with the target mesh.
        
    Outputs:
    out: np.array(N, 3), numpy array with x coordinates in column 0,
        y coordinates in column 1, and z coordinates in column 2.
    
    """
    # Create list of vertices from the vectors of each triangle in the mesh.
    vert = np.around(np.unique(input_mesh.vectors.reshape(
        [int(input_mesh.vectors.size/3), 3]), axis=0),2)
    
    # select only vertices on the ventral stomach
    vert_ventral = vert[vert[:, 0] < 9.0, :]
    # initial output array
    out = np.zeros((input_pts.shape[0], 3))
    
    # iterate over all points in y-z plane and find the closest mesh vertex in this plane
    for i, pt in enumerate(input_pts):      
        min_arg = np.argmin(np.sqrt(np.sum(np.power(
            np.abs((pt-vert_ventral[:, 1:3])),2), 1)))
        matched_x = vert_ventral[min_arg, 0]
        # Add some random movement so that triangles don't obscure points
        matched_x -= 2 - np.random.rand()*1 
        out[i, :] = [matched_x, pt[0], pt[1]]
    return out

### **Loading the 2D data**
In the 2D datasets that we are using, the distances are in percentages relative to an origin situated in the pyloric end of the stomach for the y-axis and near the oesophagus for the z-axis. We are going to convert those into millimeters instead. For this, we are going to define the limits in the z- and y-axis. 

Here is a 2D representation of the data we are going to map to the 3D mesh.
![2d](img/2d_data_viz.png)

In [32]:
# Setup maximimum y and z widths based on scale in image.
z_lims = [0, 36.7]
y_lims = [4.6, 0]

col_keeps = {'%x (distance from pylorus side)':'%x', '%y (distance from bottom)':'%y',
             'Average IGLE Area (um²)':'area', 'Area Of Innervation':'area', 
             'Neuron Area Of Innervation (um²) -Convex Hull':'area'}

We can now load the locations of the nerves into data frames:

In [33]:
df_igle = load_data('data/IGLE_data.xlsx', col_keeps, y_lims, z_lims)
df_ima = load_data('data/IMA_analyzed_data.xlsx', col_keeps, y_lims, z_lims)
df_efferent = load_data('data/Efferent_data.xlsx', col_keeps, y_lims, z_lims) 

### **Preparing the 2D data**
Now that we have loaded are datasets into Python, we are going to prepare to be mapped then plotted. For this, we are going to use our _convert_2D_data_ helper function. This will convert our data from the pandas DataFrame to a numpy array which will be more easy to manipulate for our objective.

In [34]:
efferent_2D, efferent_intensity = convert_2D_data(df_efferent)
igle_2D, igle_intensity = convert_2D_data(df_igle)
ima_2D, ima_intensity = convert_2D_data(df_ima)

### **Loading the 3D mesh**
The 2D data is all set, time to focus on the 3D mesh! Let us load the 3D stomach mesh provided in the _data_ folder for this tutorial. However, it is possible to use **SPARC** tools to generate the mesh from **FAIR** data. Give link to what tool can be used and the documentation of that tool.

In [35]:
# Load the STL files and add the vectors to the plot
stomach_mesh = msh.Mesh.from_file('data/stom_surf_mesh.stl')

# with coordinate axes (imported with signs reversed)
stomach_mesh.x -= np.min(stomach_mesh.x.flatten())
stomach_mesh.y -= np.min(stomach_mesh.y.flatten())
stomach_mesh.z -= np.min(stomach_mesh.z.flatten())

### **Mapping data**
We are now ready to map the 2D points to the 3D mesh using the helper function we have written earlier. Explain what other tools are available for mapping.

In [36]:
# Project to 3D stomach surface.
efferent_3D = map_to_3D(efferent_2D, stomach_mesh)
igle_3D = map_to_3D(igle_2D, stomach_mesh)
ima_3D = map_to_3D(ima_2D, stomach_mesh)

### **Visualising data**
The data has now been mapped to the 3D mesh. All that is left to do is visualise the mesh and the nerves! We are going to be using the matplotlib package to visualise the data. We define a plotting function to be able to create an interactive interface. We create three checkboxes, one for each dataset, in order to visulasie the points from each data set individually. The efferents are plotted in green, the IMAs are plotted in red, and the igle are plotted in blue. You can select the data by ticking or unticking the boxes. 

#### ⚠️  **SPARC Guru tip**: 
Did you know that there are some **SPARC** tools that you can use to visualise the data? They have their own GUI that you can use. For this tutorial, we decided to keep everything in one place but they also have compatible Python APIs that will allow you to integrate the tool directly into this Notebook! Take a look [here](link to tool) to read the documentation. 

In [37]:
# Enable interactivity in jupyterlab.
%matplotlib widget 

def plotting_fct(mesh, efferent_3D, igle_3D, ima_3D, efferent_intensity, 
                 igle_intensity, ima_intensity, efferent, igle, ima):
    # Start a matplotlib 3d interactive figure.
    fig = plt.figure()
    fig.suptitle('Projection visualisation', fontsize=16)
    ax = fig.add_subplot(projection='3d')

    # Add mesh as triangle polygons to 3d matplotlib view.
    faces = mplot3d.art3d.Poly3DCollection(mesh.vectors, color=(0, 0.4470, 0.7410))
    faces.set_edgecolor((0, 0.4470, 0.7410, 0.1))
    faces.set_alpha(0.1)
    ax.add_collection3d(faces)

    # Plot the projected neurons coloured by their area of innervation.
    if efferent:
        ax.scatter(efferent_3D[:,0], efferent_3D[:,1], efferent_3D[:,2], 
                   c=efferent_intensity, cmap='Greens')
        
    if ima:
        ax.scatter(ima_3D[:,0], ima_3D[:,1], ima_3D[:,2], 
                   c=ima_intensity, cmap='Reds')
        
    if igle:
        ax.scatter(igle_3D[:,0], igle_3D[:,1], igle_3D[:,2], 
                    c=igle_intensity, cmap='Blues')        


    # Scale view to the mesh size and turn off axis chrome.
    ax.set_xlim(0,40)
    ax.set_ylim(-10,30)
    ax.set_zlim(-10,30)
    ax.axis('off')

    # Show ventral surface and plot projected neurons.
    ax.view_init(10, -57, 'y')
    plt.show()

efferent=Checkbox(value=True, description="efferent")
igle=Checkbox(value=False, description="IGLE")
ima=Checkbox(value=False, description="IMA")
    
interact(plotting_fct, mesh=fixed(stomach_mesh), efferent_3D=fixed(efferent_3D), 
            igle_3D=fixed(igle_3D), ima_3D=fixed(ima_3D),
            efferent_intensity=fixed(efferent_intensity), 
            igle_intensity=fixed(igle_intensity), ima_intensity=fixed(ima_intensity), 
            efferent=efferent, igle=igle, ima=ima)    


interactive(children=(Checkbox(value=True, description='efferent'), Checkbox(value=False, description='IGLE'),…

<function __main__.plotting_fct(mesh, efferent_3D, igle_3D, ima_3D, efferent_intensity, igle_intensity, ima_intensity, efferent, igle, ima)>

### **Congratulations**
You have successfully completed your first Quilted Tutorial and are now on your way to becoming a **SPARC** Guru! 

We invite you to reuse this tutorial and explore the possibilities of using **SPARC** tools when possible. Try different things, such as adding ***your own code*** to generate a new stomach mesh with INSERT TOOL NAME. 